In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# 1. Load Data
df = pd.read_csv('/kaggle/input/tour-logs-training-dataset/tour_logs_train.csv')

In [2]:
# --- C. Handling Outliers & Errors (Updated) ---

# First: Fill NaNs with the median so comparisons don't fail
df['Crowd_Energy'] = df['Crowd_Energy'].fillna(df['Crowd_Energy'].median())
df['Volume_Level'] = df['Volume_Level'].fillna(df['Volume_Level'].median())

# Second: Clamp values to realistic ranges using .clip()
# This replaces the > and < logic and prevents warnings
df['Crowd_Energy'] = df['Crowd_Energy'].clip(0, 100)
df['Volume_Level'] = df['Volume_Level'].clip(1, 11)

# Cap Crowd Size at 1000 (Based on Venue capacity of ~800)
df['Crowd_Size'] = df['Crowd_Size'].fillna(df['Crowd_Size'].median()).clip(0, 1000)

In [3]:
def clean_data(df):
    # --- A. Currency Normalization ---
    def normalize_price(price_str):
        if pd.isna(price_str) or str(price_str).lower() == 'free':
            return 0.0
        
        # Strip text like (VIP: $...) and keep only the first numeric part
        price_str = re.split(r'\(', str(price_str))[0].strip()
        
        # Extract numeric value
        numeric_match = re.search(r"[-+]?\d*\.\d+|\d+", price_str)
        if not numeric_match: return np.nan
        val = float(numeric_match.group())
        
        # Apply Exchange Rates from PDF [cite: 173, 214]
        if '£' in price_str: return val * 1.27
        if '€' in price_str: return val * 1.09
        if 'USD' in price_str or '$' in price_str: return val
        return val # Default to USD if no symbol

    df['Ticket_Price_USD'] = df['Ticket_Price'].apply(normalize_price)

    # --- B. Date Standardization ---
    # Handle descriptive text like "Late Night" before parsing [cite: 213]
    df['Show_DateTime'] = df['Show_DateTime'].replace({'Late Night': '22:00:00', 'Evening': '19:00:00'})
    df['Clean_DateTime'] = pd.to_datetime(df['Show_DateTime'], errors='coerce')
    
    # Extract features for EDA
    df['Hour'] = df['Clean_DateTime'].dt.hour
    # Override the roadie's 'Day_of_Week' with the actual calendar day
    df['Day_of_Week_Actual'] = df['Clean_DateTime'].dt.dayofweek 

    # --- C. Outlier Removal (Based on Scribbles) ---
    # Clamp Crowd Energy to 0-100 [cite: 216]
    df.loc[df['Crowd_Energy'] > 100, 'Crowd_Energy'] = 100
    df.loc[df['Crowd_Energy'] < 0, 'Crowd_Energy'] = 0
    
    # Clamp Volume Level to 1-11 [cite: 216]
    df.loc[df['Volume_Level'] > 11, 'Volume_Level'] = 11
    df.loc[df['Volume_Level'] < 1, 'Volume_Level'] = 1
    
    # Cap Crowd Size at 1,000 (Venue capacity is ~800-1000) [cite: 187, 216]
    df.loc[df['Crowd_Size'] > 1000, 'Crowd_Size'] = 800 

    # --- D. Handling Missing Values (Impute by Venue) ---
    df['Volume_Level'] = df.groupby('Venue_ID')['Volume_Level'].transform(lambda x: x.fillna(x.median()))
    df['Crowd_Size'] = df.groupby('Venue_ID')['Crowd_Size'].transform(lambda x: x.fillna(x.median()))

    return df

df_clean = clean_data(df)